<a href="https://colab.research.google.com/github/Ayeshaaa777/Decorus/blob/main/nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

## 1. Setup

In [ ]:
import logging
logging.disable(logging.INFO)

from pprint import PrettyPrinter
pprint = PrettyPrinter(compact=True).pprint

!pip3 install -q datasets transformers evaluate accelerate

import transformers
import torch
import evaluate
import accelerate
from collections import defaultdict
from transformers import BertTokenizer, BertModel

!pip install optuna
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.3 MB/s eta 0:00:00


---

## 2. Data download, sampling and preprocessing

### 2.1. Download the corpus

In [ ]:
import datasets

dataset = datasets.load_dataset("mteb/amazon_reviews_multi")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mteb/amazon_reviews_multi contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mteb/amazon_reviews_multi
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

### 2.2. Sampling and preprocessing

In [ ]:
print (dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 1200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 30000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 30000
    })
})


In [ ]:
pprint(dataset['test'][ 5000 ])

{'id': 'en_0199937',
 'label': 0,
 'label_text': '0',
 'text': 'Don’t waste your time!\n'
         '\n'
         'These are AWFUL. They are see through, the fabric feels like '
         'tablecloth, and they fit like children’s clothing. Customer service '
         'did seem to be nice though, but I regret missing my return date for '
         'these. I wouldn’t even donate them because the quality is so poor.'}


In [ ]:
# each language has 210K rows, 200K train, 5K val, 5K test

de_train=dataset["train"].select(range(200000 - 1))
de_val=dataset["validation"].select(range(5000 - 1))
de_test=dataset["test"].select(range(5000 - 1))

en_train=dataset["train"].select(range(200000, 200000 + 200000 -1))
en_val=dataset["validation"].select(range(5000, 5000 + 5000 -1))
en_test=dataset["test"].select(range(5000, 5000 + 5000 -1))

In [ ]:
# down sampling

train_size = 10000
val_size = 1000
test_size = 1000

de_train=de_train.shuffle()
de_test=de_test.shuffle()
de_val=de_val.shuffle()

en_train=en_train.shuffle()
en_test=en_test.shuffle()
en_val=en_val.shuffle()

In [ ]:
de_train=de_train.select(range(train_size))
de_test=de_test.select(range(test_size))
de_val=de_val.select(range(val_size))

en_train=en_train.select(range(train_size))
en_test=en_test.select(range(test_size))
en_val=en_val.select(range(val_size))

In [ ]:
en_dataset = datasets.DatasetDict({'train': en_train, 'validation': en_val, 'test': en_test})
de_dataset = datasets.DatasetDict({'train': de_train, 'validation': de_val, 'test': de_test})

merge_dataset = datasets.DatasetDict({'train' : datasets.concatenate_datasets([en_train, de_train]),
                                      'validation' : datasets.concatenate_datasets([en_val, de_val]),
                                      'test' : datasets.concatenate_datasets([en_test, de_test])
                                      })
merge_dataset.shuffle()

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 2000
    })
})

---

## 3. Machine learning model

### 3.1. Model training

In [ ]:
#  code to train the transformer based model on the training set and evaluate the performance on the validation set here

model_name = "bert-base-multilingual-cased"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)  # TODO: check this

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize(example):
    return tokenizer(
        example["text"],
        max_length=512,
        truncation=True,
        padding='max_length'
    )

# Apply the tokenizer to the whole dataset using .map()
en_dataset = en_dataset.map(tokenize)
de_dataset = de_dataset.map(tokenize)
merge_dataset = merge_dataset.map(tokenize)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
trainer_args = transformers.TrainingArguments(
    "checkpoints",
    evaluation_strategy="steps",
    logging_strategy="steps",
    load_best_model_at_end=True,
    eval_steps=100,
    logging_steps=100,
    learning_rate=0.000001,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    max_steps=1000,
)

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels
    predictions = outputs.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)


data_collator = transformers.DataCollatorWithPadding(tokenizer)

early_stopping = transformers.EarlyStoppingCallback(
    early_stopping_patience=5
)

In [ ]:
class LogSavingCallback(transformers.TrainerCallback):
    def on_train_begin(self, *args, **kwargs):
        self.logs = defaultdict(list)
        self.training = True

    def on_train_end(self, *args, **kwargs):
        self.training = False

    def on_log(self, args, state, control, logs, model=None, **kwargs):
        if self.training:
            for k, v in logs.items():
                if k != "epoch" or v not in self.logs[k]:
                    self.logs[k].append(v)

training_logs = LogSavingCallback()

**baseline** : Train on English --> Evaluate on English

In [ ]:
en_trainer = None
en_trainer = transformers.Trainer(
    model=model,
    args=trainer_args,
    train_dataset=en_dataset['train'],
    eval_dataset=en_dataset['validation'],
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer = tokenizer,
    callbacks=[early_stopping, training_logs]
)

en_trainer.train()


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
100,1.614900,1.611624,0.245000
200,1.599000,1.597163,0.297000
300,1.587300,1.575503,0.327000
400,1.567700,1.552336,0.350000
500,1.555600,1.528499,0.365000
600,1.520500,1.502784,0.370000
700,1.489900,1.482036,0.377000
800,1.484000,1.466479,0.374000
900,1.470900,1.458257,0.376000
1000,1.462700,1.454586,0.375000


TrainOutput(global_step=1000, training_loss=1.5352706451416016, metrics={'train_runtime': 1159.4031, 'train_samples_per_second': 6.9, 'train_steps_per_second': 0.863, 'total_flos': 2104945139712000.0, 'train_loss': 1.5352706451416016, 'epoch': 0.8})

### 3.2 Hyperparameter optimization

In [ ]:
LR_MIN = 4e-5
LR_CEIL = 0.01
WD_MIN = 4e-5
WD_CEIL = 0.01
MIN_EPOCHS = 2
MAX_EPOCHS = 5
PER_DEVICE_EVAL_BATCH = 8
PER_DEVICE_TRAIN_BATCH = 8
NUM_TRIALS = 1
SAVE_DIR = 'opt-test'
MAX_LENGTH = 512

In [ ]:
def objective(trial: optuna.Trial):

  training_args = transformers.TrainingArguments(
        output_dir=SAVE_DIR,
        learning_rate=trial.suggest_loguniform('learning_rate', low=LR_MIN, high=LR_CEIL),
        weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),
        num_train_epochs=trial.suggest_int('num_train_epochs', low = MIN_EPOCHS,high = MAX_EPOCHS),
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH,
        per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
        disable_tqdm=True)

  trainer = transformers.Trainer(
      model=model,
      args=training_args,
      train_dataset=en_dataset['train'],
      eval_dataset=en_dataset['validation'])

  result = trainer.train()
  return result.training_loss

In [ ]:
#  code for hyperparameter optimization here
study = optuna.create_study(study_name='hp-search-electra', direction='maximize', sampler=optuna.samplers.RandomSampler())
study.optimize(func=objective, n_trials=NUM_TRIALS)

<ipython-input-14-37c2f150d060>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', low=LR_MIN, high=LR_CEIL),
<ipython-input-14-37c2f150d060>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay=trial.suggest_loguniform('weight_decay', WD_MIN, WD_CEIL),


{'loss': 1.7797, 'learning_rate': 0.0010860928614771352, 'epoch': 0.4}
{'loss': 1.7412, 'learning_rate': 0.0009190016520191142, 'epoch': 0.8}
{'loss': 1.7327, 'learning_rate': 0.0007519104425610935, 'epoch': 1.2}
{'loss': 1.6965, 'learning_rate': 0.0005848192331030727, 'epoch': 1.6}
{'loss': 1.6894, 'learning_rate': 0.0004177280236450519, 'epoch': 2.0}
{'loss': 1.6638, 'learning_rate': 0.00025063681418703115, 'epoch': 2.4}
{'loss': 1.6417, 'learning_rate': 8.354560472901038e-05, 'epoch': 2.8}
{'train_runtime': 3017.1813, 'train_samples_per_second': 9.943, 'train_steps_per_second': 1.243, 'train_loss': 1.7010922688802084, 'epoch': 3.0}


In [ ]:
print(study.best_value)
print(study.best_params)
print(study.best_trial)

1.7010922688802084
{'learning_rate': 0.0012531840709351558, 'weight_decay': 0.0002118427006962042, 'num_train_epochs': 3}
FrozenTrial(number=0, state=TrialState.COMPLETE, values=[1.7010922688802084], datetime_start=datetime.datetime(2024, 1, 14, 14, 58, 22, 119747), datetime_complete=datetime.datetime(2024, 1, 14, 15, 48, 40, 566281), params={'learning_rate': 0.0012531840709351558, 'weight_decay': 0.0002118427006962042, 'num_train_epochs': 3}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.01, log=True, low=4e-05, step=None), 'weight_decay': FloatDistribution(high=0.01, log=True, low=4e-05, step=None), 'num_train_epochs': IntDistribution(high=5, log=False, low=2, step=1)}, trial_id=0, value=None)


**With Best Params**

In [ ]:
best_trainer_args = transformers.TrainingArguments(
    "checkpoints",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy='epoch',
    load_best_model_at_end=True,
    eval_steps=100,
    logging_steps=100,
    learning_rate=0.00125,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    max_steps=1000,
    num_train_epochs=3,
    weight_decay= 0.00021
)


In [ ]:
best_en_trainer = None
best_en_trainer = transformers.Trainer(
    model=model,
    args=best_trainer_args,
    train_dataset=en_dataset['train'],
    eval_dataset=en_dataset['validation'],
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer = tokenizer,
    callbacks=[early_stopping, training_logs]
)

best_en_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,1.664500,1.619073,0.185000


TrainOutput(global_step=1000, training_loss=1.66446826171875, metrics={'train_runtime': 815.9068, 'train_samples_per_second': 9.805, 'train_steps_per_second': 1.226, 'total_flos': 2104945139712000.0, 'train_loss': 1.66446826171875, 'epoch': 0.8})

### 3.3. Evaluation on test set

In [ ]:
# code to evaluate the final model on the test set here
en_eval_results = en_trainer.evaluate(en_dataset["test"])

pprint(en_eval_results)

print('Accuracy:', en_eval_results['eval_accuracy'])

{'epoch': 0.8,
 'eval_accuracy': 0.449,
 'eval_loss': 1.2808046340942383,
 'eval_runtime': 33.8751,
 'eval_samples_per_second': 29.52,
 'eval_steps_per_second': 0.945}
Accuracy: 0.449


### 3.4. Multilingual and cross-lingual experiments

**Multilingual**

In [ ]:
de_en_trainer = None
de_en_trainer = transformers.Trainer(
    model=model,
    args=trainer_args,
    train_dataset=merge_dataset['train'],
    eval_dataset=merge_dataset['validation'],
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer = tokenizer,
    callbacks=[early_stopping, training_logs]
)

de_en_trainer.train()

Step,Training Loss,Validation Loss,Accuracy
100,1.313600,1.302866,0.407000
200,1.299300,1.279573,0.425000
300,1.309200,1.269392,0.443500
400,1.266600,1.256128,0.444500
500,1.280400,1.250214,0.436500
600,1.276600,1.241823,0.449000
700,1.257400,1.230625,0.454000
800,1.207000,1.226553,0.454500
900,1.252400,1.224106,0.456000
1000,1.215900,1.223727,0.456500


TrainOutput(global_step=1000, training_loss=1.2678352508544921, metrics={'train_runtime': 1483.01, 'train_samples_per_second': 5.394, 'train_steps_per_second': 0.674, 'total_flos': 2104945139712000.0, 'train_loss': 1.2678352508544921, 'epoch': 0.4})

In [ ]:
de_en_eval_results = de_en_trainer.evaluate(en_dataset["test"])

pprint(de_en_eval_results)

print('Accuracy:', de_en_eval_results['eval_accuracy'])

{'epoch': 0.4,
 'eval_accuracy': 0.454,
 'eval_loss': 1.2270537614822388,
 'eval_runtime': 36.0045,
 'eval_samples_per_second': 27.774,
 'eval_steps_per_second': 0.889}
Accuracy: 0.454


**Crosslingual**

In [ ]:
de_trainer = None
de_trainer = transformers.Trainer(
    model=model,
    args=trainer_args,
    train_dataset=de_dataset['train'],
    eval_dataset=de_dataset['validation'],
    compute_metrics=compute_accuracy,
    data_collator=data_collator,
    tokenizer = tokenizer,
    callbacks=[early_stopping, training_logs]
)

de_trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
100,1.612700,1.606141,0.208000
200,1.609800,1.600255,0.205000
300,1.605300,1.594637,0.244000
400,1.591400,1.583135,0.324000
500,1.580300,1.572298,0.344000
600,1.578400,1.561795,0.370000
700,1.567700,1.551601,0.378000
800,1.565300,1.544444,0.377000
900,1.551700,1.539004,0.386000
1000,1.550300,1.536725,0.389000


TrainOutput(global_step=1000, training_loss=1.581300308227539, metrics={'train_runtime': 1151.5527, 'train_samples_per_second': 6.947, 'train_steps_per_second': 0.868, 'total_flos': 2104945139712000.0, 'train_loss': 1.581300308227539, 'epoch': 0.8})

In [ ]:
de_eval_results = de_trainer.evaluate(en_dataset["test"])

pprint(de_eval_results)

print('Accuracy:', de_eval_results['eval_accuracy'])

{'epoch': 0.8,
 'eval_accuracy': 0.313,
 'eval_loss': 1.569287896156311,
 'eval_runtime': 34.8595,
 'eval_samples_per_second': 28.687,
 'eval_steps_per_second': 0.918}
Accuracy: 0.313






### 4. Data Selection and Sentence representations

In [ ]:
# code to create a sentence embedding for the given text here

#each label has 20% of the training data, to get the label 'm' I need  200000 + int(m * label_range) th data
label_range = 200000/5
en_sen = []
for i in range(5):
  en_sen.append(dataset['train'][ 200000 + int(i * label_range)]['text'])
en_sen

["I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review ...\n\nArrived broken. Manufacturer defect. Two of the legs of the base were not completely formed, so there was no way to insert the casters. I unpackaged the entire chair and hardware before noticing this. So, I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review of part of a chair I never got to sit in. I will go so far as to include a picture of what their injection molding and quality assurance process missed though. I will be hesitant to buy again. It makes me wonder if there aren't missing structures and supports that don't impede the assembly process.",
 "Ineffective\n\nDidn't work. The heating element was too small. We had better luck with a faux-fur heated blanket.",
 "Waterproof case was broken\n\nIt's a very nice kit, it came with all the accessories, BUT my waterproof case was broken, the thing that closes it wa

In [ ]:
label_range = 200000/5
de_sen = []
for j in range(20):
  for i in range(5):
    de_sen.append(dataset['train'][ j + int(i * label_range)]['text'])

**I extracted 100 sentences in German but google colab craches when I try to encode it. It goes out of RAM. So I use a smaller set.**

In [ ]:
label_range = 200000/5
de_sen = []
for j in range(5):
  for i in range(5):
    de_sen.append(dataset['train'][ j + int(i * label_range)]['text'])

In [ ]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
# Tokenize the sentences
tokens = tokenizer(en_sen, return_tensors='pt', padding=True, truncation=True)

# Forward pass to obtain embeddings
outputs = model(**tokens)
en_encodes = outputs.pooler_output.squeeze(0)

tokens = tokenizer(de_sen, return_tensors='pt', padding=True, truncation=True)
outputs = model(**tokens)
de_encodes = outputs.pooler_output.squeeze(0)

In [ ]:
print(en_encodes.dim())
print(type(en_encodes))

2
<class 'torch.Tensor'>


### 4.1 Cosine similarity

In [ ]:
#  code to calculate the cosine similarity of the embeddings and select the target sentence that maximizes the cosine similarity here
import torch.nn.functional as F
# Calculate the cosine similarity for each target sentence
max_similarity = float('-inf')
max_similarity_index = None

for j, en_s in enumerate(en_encodes):

  max_similarity = float('-inf')
  max_similarity_index = None

  for i, de_s in enumerate(de_encodes):

      cos = torch.nn.CosineSimilarity(dim=0)
      similarity = cos(en_s, de_s)

      # Update max similarity and index
      if similarity > max_similarity:
          max_similarity = similarity
          max_similarity_index = i

  # Print the target sentence with the maximum cosine similarity
  print("** Reference sentence:", en_sen[j])
  print("** Target sentence with max cosine similarity:", de_sen[max_similarity_index])
  print("Cosine Similarity:", max_similarity)
  print("-----------")

** Reference sentence: I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review ...

Arrived broken. Manufacturer defect. Two of the legs of the base were not completely formed, so there was no way to insert the casters. I unpackaged the entire chair and hardware before noticing this. So, I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review of part of a chair I never got to sit in. I will go so far as to include a picture of what their injection molding and quality assurance process missed though. I will be hesitant to buy again. It makes me wonder if there aren't missing structures and supports that don't impede the assembly process.
** Target sentence with max cosine similarity: Eleaf iStick Pico

Habe leider den originalen Drip Tip geschrottet und mir dann einen der dem Original ähnlich sah bestellt, diesen habe ich verloren, der so locker saß. Aufgrund der bisherigen Bewertunge